In [53]:
def get_disaster_declaration(state: str, declarationType: str, limit: int = 10) -> list:
    """
    Retrieve disaster declarations from FEMA's Open API for a specific state.

    Args:
        state (str): Two-letter state abbreviation (e.g., 'TX' for Texas)
        declarationType (str): Type of declaration (e.g., 'DR' for major disaster)
        limit (int, optional): Maximum number of results to return. Defaults to 10.

    Returns:
        list: Formatted disaster declaration summaries or error message
    """
    base_url = "https://www.fema.gov/api/open/v2/DisasterDeclarationsSummaries"

    # Parameters for the API request
    params = {
        "state": state,                     # Filter by state (e.g., Texas)
        "declarationType": declarationType, # Declaration type (e.g., DR for major disaster)
    }

    try:
        # Make the request
        response = requests.get(base_url, params=params)
        response.raise_for_status()  # Raise an exception for bad status codes

        # Convert response to JSON
        data = response.json()
        disaster_summaries = data.get("DisasterDeclarationsSummaries", [])



        # Format each disaster entry as a string
        formatted_disasters = []
        for disaster in disaster_summaries:

            if state== disaster.get('state'):
                formatted_disasters.append(
                    [f"Disaster ID: {disaster.get('disasterNumber')}\n"
                    f"Declaration String: {disaster.get('femaDeclarationString')}\n"
                    f"State: {disaster.get('state')}\n"
                    f"Declaration Type: {disaster.get('declarationType')}\n"
                    f"Title: {disaster.get('declarationTitle')}\n"
                    f"Incident Type: {disaster.get('incidentType')}\n"
                    f"Declaration Date: {disaster.get('declarationDate')}\n"
                    f"Incident Begin Date: {disaster.get('incidentBeginDate')}\n"
                    f"Incident End Date: {disaster.get('incidentEndDate')}\n"
                    f"Programs Declared: "
                    f"IH: {disaster.get('ihProgramDeclared')}, "
                    f"IA: {disaster.get('iaProgramDeclared')}, "
                    f"PA: {disaster.get('paProgramDeclared')}, "
                    f"HM: {disaster.get('hmProgramDeclared')}\n"
                    f"Region: {disaster.get('region')}\n"
                    f"Last Refresh: {disaster.get('lastRefresh')}\n"]
                )

        print(len(formatted_disasters[-limit:]))

        formatted_disasters = formatted_disasters[-limit:]

        result = ""

        for disaster in formatted_disasters:
            result += f"{disaster[0]}\n"

        return result
    except requests.RequestException as e:
        return [f"Failed to retrieve data: {e}"]

In [54]:
data = get_disaster_declaration(state="TX", declarationType="DR", limit=3)
# Print each disaster declaration
print(data)

3
Disaster ID: 4798
Declaration String: DR-4798-TX
State: TX
Declaration Type: DR
Title: HURRICANE BERYL 
Incident Type: Hurricane
Declaration Date: 2024-07-09T00:00:00.000Z
Incident Begin Date: 2024-07-05T00:00:00.000Z
Incident End Date: 2024-07-09T00:00:00.000Z
Programs Declared: IH: True, IA: False, PA: True, HM: True
Region: 6
Last Refresh: 2024-08-27T18:22:14.800Z

Disaster ID: 4798
Declaration String: DR-4798-TX
State: TX
Declaration Type: DR
Title: HURRICANE BERYL 
Incident Type: Hurricane
Declaration Date: 2024-07-09T00:00:00.000Z
Incident Begin Date: 2024-07-05T00:00:00.000Z
Incident End Date: 2024-07-09T00:00:00.000Z
Programs Declared: IH: True, IA: False, PA: True, HM: True
Region: 6
Last Refresh: 2024-08-27T18:22:14.800Z

Disaster ID: 4798
Declaration String: DR-4798-TX
State: TX
Declaration Type: DR
Title: HURRICANE BERYL 
Incident Type: Hurricane
Declaration Date: 2024-07-09T00:00:00.000Z
Incident Begin Date: 2024-07-05T00:00:00.000Z
Incident End Date: 2024-07-09T00:00:00

In [12]:
# API endpoint and key
api_url = "https://www.fema.gov/api/open/v2/DisasterDeclarationsSummaries"
params = {
    "page[size]": 10,  # Number of results per page
    "sort": "declarationDate",  # Sort by declaration date
}

try:
    # Make the request
    response = requests.get(api_url, params=params)
    response.raise_for_status()  # Raise exception for HTTP errors

    # Parse JSON data
    disaster_data = response.json()
    print(disaster_data)
    # for disaster in disaster_data['DisasterDeclarationsSummaries']:
    #     print(f"Disaster: {disaster['title']}, Date: {disaster['declarationDate']}")
except requests.exceptions.RequestException as e:
    print(f"Error fetching data: {e}")

{'metadata': {'skip': 0, 'filter': '', 'orderby': '', 'select': None, 'rundate': '2024-11-29T04:36:47.439Z', 'top': 1000, 'format': 'json', 'metadata': True, 'entityname': 'DisasterDeclarationsSummaries', 'version': 'v2', 'url': '/api/open/v2/DisasterDeclarationsSummaries', 'count': 0}, 'DisasterDeclarationsSummaries': [{'femaDeclarationString': 'FM-5530-NV', 'disasterNumber': 5530, 'state': 'NV', 'declarationType': 'FM', 'declarationDate': '2024-08-12T00:00:00.000Z', 'fyDeclared': 2024, 'incidentType': 'Fire', 'declarationTitle': 'GOLD RANCH FIRE', 'ihProgramDeclared': False, 'iaProgramDeclared': False, 'paProgramDeclared': True, 'hmProgramDeclared': True, 'incidentBeginDate': '2024-08-11T00:00:00.000Z', 'incidentEndDate': None, 'disasterCloseoutDate': None, 'tribalRequest': False, 'fipsStateCode': '32', 'fipsCountyCode': '031', 'placeCode': '99031', 'designatedArea': 'Washoe (County)', 'declarationRequestNumber': '24123', 'lastIAFilingDate': None, 'incidentId': '2024081201', 'region'

In [ ]:
import us
def get_weather_alerts(state:str):
    state = us.states.lookup(state)
    if state:
        url = f"https://api.weather.gov/alerts/active?area={state.abbr.upper()}"
        try:
            response = requests.get(url)
            response.raise_for_status()  # Check for HTTP errors
            
            alerts_data = response.json()
            
            # Check if there are any alerts
            if alerts_data.get("features"):
                return alerts_data["features"]
            else:
                return f"No active alerts for {state.abbr.upper()}."

        except requests.exceptions.RequestException as e:
            return f"An error occurred: {e}"
    return "A state by this name doesn't exist in USA"


In [ ]:
state = "TX"  # Replace with any two-letter state abbreviation
alerts = get_weather_alerts(state)
print(alerts)

In [1]:
import requests

def is_in_evacuation_zone(state: str,
                          latitude: float,
                          longitude: float) -> str:

    # Define query parameters
    params = {
        "f": "json",
        "geometry": "",
        "geometryType": "",
        "spatialRel": "",
        "outFields": "*",
        "where": "1=1",
        "defaultSR": 102100
    }

    if state == "FL":
        base_url = "https://services.arcgis.com/3wFbqsFPLeKqOlIK/arcgis/rest/services/KYZ_ZL_Vector_Enriched_Calculated_20230608/FeatureServer/28/query"
        params["geometry"] = f"{latitude},{longitude}"
        params["geometryType"] = "esriGeometryPoint"
        params["spatialRel"] = "esriSpatialRelWithin"


    elif state == "TX":
        base_url = "https://services.arcgis.com/su8ic9KbA7PYVxPS/arcgis/rest/services/HurricaneEvac_Zones/FeatureServer/0/query"

        offset = 10000

        xmin = latitude - offset
        ymin = longitude - offset
        xmax = latitude + offset
        ymax = longitude + offset

        params["geometry"] = f"{xmin},{ymin},{xmax},{ymax}"
        params["geometryType"] = "esriGeometryEnvelope"
        params["spatialRel"] = "esriSpatialRelIntersects"

    # Make the request
    response = requests.get(base_url, params=params)
    # Check if the request was successful
    if response.status_code == 200:
        data = response.json()
        zone_features = data.get("features", [])

        if zone_features:
            res_zones = set()
            for feature in zone_features:
                attributes = feature["attributes"]
                zone = attributes.get("Zone") or attributes.get("EZone")
                #status = attributes.get("STATUS", '')
                res_zones.add(zone)

            return f"Your location is in Evacuation Zone(s) {zone}."
        else:
            return "The location is not within an evacuation zone."
    else:
        return f"Failed to retrieve data: {response.status_code}"

latitude =  -9060471.185883742 #fl
longitude = 3005976.5677590542 #fl

latitude =  -10603790.548624992 #tx
longitude =  3485082.5049009863 #tx

latitude =  -9064369.772908427
longitude = 3009501.0313601447

data = is_in_evacuation_zone(state="FL", latitude=latitude, longitude=longitude)
print(data)

Your location is in Evacuation Zone(s) D.


In [22]:
state = "TX"  # Replace with any two-letter state abbreviation
alerts = get_weather_alerts(state)
print(alerts)

[{'id': 'https://api.weather.gov/alerts/urn:oid:2.49.0.1.840.0.60ed8d125ec49c6008b416d60c302c2f0961fb53.001.1', 'type': 'Feature', 'geometry': None, 'properties': {'@id': 'https://api.weather.gov/alerts/urn:oid:2.49.0.1.840.0.60ed8d125ec49c6008b416d60c302c2f0961fb53.001.1', '@type': 'wx:Alert', 'id': 'urn:oid:2.49.0.1.840.0.60ed8d125ec49c6008b416d60c302c2f0961fb53.001.1', 'areaDesc': 'Coastal Aransas; Coastal Kleberg; Coastal Nueces; Coastal San Patricio; Coastal Refugio; Coastal Calhoun', 'geocode': {'SAME': ['048007', '048273', '048355', '048409', '048391', '048057'], 'UGC': ['TXZ245', 'TXZ342', 'TXZ343', 'TXZ344', 'TXZ346', 'TXZ347']}, 'affectedZones': ['https://api.weather.gov/zones/forecast/TXZ245', 'https://api.weather.gov/zones/forecast/TXZ342', 'https://api.weather.gov/zones/forecast/TXZ343', 'https://api.weather.gov/zones/forecast/TXZ344', 'https://api.weather.gov/zones/forecast/TXZ346', 'https://api.weather.gov/zones/forecast/TXZ347'], 'references': [{'@id': 'https://api.weat

In [ ]:
from utils import arcgis_to_gmaps, gmaps_to_arcgis, get_distance_google_maps
import requests

def find_nearest_shelter_texas(
                    latitude: float,
                    longitude: float,
                    resCount: int = 3) -> str:
    
    """
    Retrieves information about the nearest shelter based on a given location in Texas.

    Parameters:
    ----------
    latitude : float
        Latitude of the user's location.
    longitude : float
        Longitude of the user's location.

    Returns:
    --------
    dict
        Details of the nearest shelter, including name, address, and available services, or an error message if no shelters are found.

    Example:
    --------
    >>> get_nearest_shelter(27.994402, -81.760254)
    'Name: Central Shelter | 123 Main St, City, State'
    """

    # API Endpoint
    base_url = "https://services5.arcgis.com/Rvw11bGpzJNE7apK/ArcGIS/rest/services/Warming_Centers_Public_View/FeatureServer/7/queryTopFeatures"
    GOOGLE_MAPS_API_KEY = ""  # Replace with your API key

    geometry_user_arcgis = (gmaps_to_arcgis(latitude, longitude))

    params = {
        "where": "Status = 'Open'", 
        "topFilter":f"""
            {{
                "groupByFields": "ShelterName",
                "topCount": {resCount},
                "orderByFields": "ShelterName"
            }}
        """,  
        "geometry": f"{geometry_user_arcgis[0]},{geometry_user_arcgis[1]}",  
        "geometryType": "esriGeometryPoint", 
        "spatialRel": "esriSpatialRelIntersects",  
        "distance": 50,  
        "units": "esriSRUnit_StatuteMile",  
        "outFields": "*",  
        "returnGeometry": "true",  
        "resultRecordCount": {resCount},  
        "f": "json"
    }

    response = requests.get(base_url, params=params)

    if response.status_code == 200:
        data = response.json()

        if "features" in data:
            distance_dict = {}
            for feature in data["features"]:
                attributes = feature['attributes']
                geometry_feat_gmaps = arcgis_to_gmaps(feature['geometry']['x'], feature['geometry']['y'])
                dist_dur = get_distance_google_maps(GOOGLE_MAPS_API_KEY, origin = f"{latitude},{longitude}", destination=f"{geometry_feat_gmaps[0]},{geometry_feat_gmaps[1]}")
                if dist_dur:
                    distance_dict[attributes['OBJECTID']] = dist_dur

            distance_dict_sorted = dict(sorted(distance_dict.items(), key=lambda x: x[1][0]))

            # Create the result list based on dist_dict's order
            res = []
            for obj_id in distance_dict_sorted.keys():
                matching_feat = next((feat['attributes'] for feat in list(data["features"]) if feat['attributes']['OBJECTID'] == obj_id), None)
                if matching_feat:
                    address = " ".join([matching_feat[add] for add in ['Address', 'Address2', 'City', 'State', 'Zip'] if matching_feat[add]])
                    """
                    # table form
                    fields = [
                        matching_feat["ShelterName"],
                        address,
                        f"{distance_dict_sorted[obj_id][0]} miles",
                        f"{distance_dict_sorted[obj_id][1]} min",
                        matching_feat.get("Hours"),
                        matching_feat.get("Phone"),
                        matching_feat.get("Website"),
                        matching_feat.get("POD_Status"),
                        matching_feat.get("AllowsAnimals"),
                        matching_feat.get("AnimalNotes"),
                    ]

                    # Filter out None values
                    fields = [field for field in fields if field]

                    # Combine fields into a compact string
                    output = " | ".join(fields) """
                    
                    fields = [
                        f"Name: {matching_feat['ShelterName']}",
                        f"{address}",
                        f"Distance: {distance_dict_sorted[obj_id][0]} miles - {distance_dict_sorted[obj_id][1]} min",
                        f"{matching_feat.get('Hours').replace('day','')}" if matching_feat.get('Hours') else None,
                        f"Contact: {matching_feat.get('Phone')}, {matching_feat.get('Website')}" 
                        if matching_feat.get('Phone') and matching_feat.get('Website') 
                        else f"Contact: {matching_feat.get('Phone')}" 
                        if matching_feat.get('Phone') 
                        else f"Website: {matching_feat.get('Website')}" 
                        if matching_feat.get('Website') 
                        else None,
                        f"POD Status: {matching_feat.get('POD_Status')}" if matching_feat.get('POD_Status') else None,
                        f"Animals: {matching_feat.get('AllowsAnimals')},{matching_feat.get('AnimalNotes')}" 
                        if matching_feat.get('AllowsAnimals') and matching_feat.get('AnimalNotes') 
                        else f"Animals: {matching_feat.get('AllowsAnimals')}" 
                        if matching_feat.get('AllowsAnimals') 
                        else f"Animals: {matching_feat.get('AnimalNotes')}" 
                        if matching_feat.get('AnimalNotes')
                        else None,
                        f"Additional Info: {matching_feat.get('Additional_Info')}" if matching_feat.get('Additional_Info') else None,
                    ]

                    output = " | ".join([field for field in fields if field])
                    res.append(output)
            if res:
                return "\n".join(res)
    else:
        f"Failed to retrieve data: {response.status_code}"

    return "No shelters found within 50 miles"

latitude = 31.792484817288468
longitude = -106.5316860128557

print(find_nearest_shelter_texas(latitude, longitude))

#this output is 716 chars

Name: Galatzan Rec Center | 650 Wallenberg Dr El Paso TX 79912 | Distance: 4.5 miles - 8.0 min | Mon through Satur 9am to 8pm and Sun 11am to 8pm | Contact: (915) 212-0734, https://www.facebook.com/elpasotexasgov | POD Status: Inactive | Animals: Yes,Service Pets Only 
Name: Chalio Acosta Sport Center  | 4321 Delta Dr El Paso TX 79905 | Distance: 8.1 miles - 11.0 min | Mon through Satur 9am to 8pm and Sun 11am to 8pm | Animals: Yes,Service Pet Only 
Name: Marty Robbins Rec Center | 11600 Vista Del Sol El Paso TX 79936 | Distance: 17.5 miles - 23.0 min | Mon through Satur 9am to 8pm and Sun 11am to 8pm | Website: https://www.facebook.com/elpasotexasgov | POD Status: Inactive | Animals: Yes,Service Pets Only


In [ ]:
from dotenv import load_dotenv
load_dotenv()

API_KEY = """

In [55]:
from googleplaces import GooglePlaces, types, lang 
import requests 
import json 
import googlemaps

# Initialising the GooglePlaces constructor
google_places = GooglePlaces(API_KEY)
gmaps = googlemaps.Client(key=API_KEY)
geocode_result = gmaps.geocode('6636 W William cannon drive')


if geocode_result:

    lat = geocode_result[0]['geometry']['location']['lat']
    lng = geocode_result[0]['geometry']['location']['lng']

    query_result = google_places.nearby_search(
            # lat_lng ={'lat': 46.1667, 'lng': -1.15},
            lat_lng ={'lat': lat, 'lng': lng},
            radius = 5000,
            # types =[types.TYPE_HOSPITAL] or
            # [types.TYPE_CAFE] or [type.TYPE_BAR]
            # or [type.TYPE_CASINO])
            types =[types.TYPE_HOSPITAL])

    # If any attributions related
    # with search results print them
    if query_result.has_attributions: 
        print (query_result.html_attributions)

    # Iterate over the search results
    for place in query_result.places:
        print(place)
        # place.get_details()
        print (place.name)
        place_geocoded = gmaps.reverse_geocode((place.geo_location['lat'], place.geo_location['lng']))
        if place_geocoded:
            print("Address",place_geocoded[0]["formatted_address"])
        else:
            print("Address Unvailable")
        print("Latitude", place.geo_location['lat'])
        print("Longitude", place.geo_location['lng'])
        print()

else:
    print("Incomplete Address! Please provide a Complete Address")

<Place name="NextCare Urgent Care Austin", lat=30.2284368, lng=-97.8634713>
NextCare Urgent Care Austin
Address 6001 W William Cannon Dr #302, Austin, TX 78749, USA
Latitude 30.2284368
Longitude -97.8634713

<Place name="Southwest Medical Village", lat=30.24475580000001, lng=-97.85745199999999>
Southwest Medical Village
Address 5625 Eiger Rd, Austin, TX 78735, USA
Latitude 30.24475580000001
Longitude -97.85745199999999

<Place name="HNI Healthcare", lat=30.2538201, lng=-97.8706287>
HNI Healthcare
Address 7500 Rialto Blvd suite 1 140, Austin, TX 78735, USA
Latitude 30.2538201
Longitude -97.8706287

<Place name="Ascension Seton Southwest Hospital", lat=30.226886, lng=-97.8918415>
Ascension Seton Southwest Hospital
Address 7900 Ranch to Market Rd 1826, Austin, TX 78737, USA
Latitude 30.226886
Longitude -97.8918415

<Place name="Andres Montanez-Flores, MD", lat=30.2538201, lng=-97.8706287>
Andres Montanez-Flores, MD
Address 7500 Rialto Blvd suite 1 140, Austin, TX 78735, USA
Latitude 30.25

In [ ]:
# Initialising the GooglePlaces constructor
google_places = GooglePlaces(API_KEY)
gmaps = googlemaps.Client(key=API_KEY)

addressvalidation_result =  gmaps.addressvalidation(['24 kelley ct'], 
                                                    regionCode='US')

In [24]:
print(addressvalidation_result)

{'result': {'verdict': {'inputGranularity': 'PREMISE', 'validationGranularity': 'ROUTE', 'geocodeGranularity': 'ROUTE', 'addressComplete': True, 'hasUnconfirmedComponents': True, 'hasInferredComponents': True}, 'address': {'formattedAddress': '24 Kelley Court, Libertyville, IL 60048, USA', 'postalAddress': {'regionCode': 'US', 'languageCode': 'en', 'postalCode': '60048', 'administrativeArea': 'IL', 'locality': 'Libertyville', 'addressLines': ['24 Kelley Ct']}, 'addressComponents': [{'componentName': {'text': '24'}, 'componentType': 'street_number', 'confirmationLevel': 'UNCONFIRMED_BUT_PLAUSIBLE'}, {'componentName': {'text': 'Kelley Court', 'languageCode': 'en'}, 'componentType': 'route', 'confirmationLevel': 'CONFIRMED'}, {'componentName': {'text': 'USA', 'languageCode': 'en'}, 'componentType': 'country', 'confirmationLevel': 'CONFIRMED'}, {'componentName': {'text': '60048'}, 'componentType': 'postal_code', 'confirmationLevel': 'CONFIRMED', 'inferred': True}, {'componentName': {'text'

In [58]:
def get_nearest_hospital(address:str,limit:int):
    # Initialising the GooglePlaces constructor

    result = ""
    
    google_places = GooglePlaces(API_KEY)
    gmaps = googlemaps.Client(key=API_KEY)
    geocode_result = gmaps.geocode(address)


    if geocode_result:

        lat = geocode_result[0]['geometry']['location']['lat']
        lng = geocode_result[0]['geometry']['location']['lng']

        query_result = google_places.nearby_search(
                # lat_lng ={'lat': 46.1667, 'lng': -1.15},
                lat_lng ={'lat': lat, 'lng': lng},
                radius = 5000,
                # types =[types.TYPE_HOSPITAL] or
                # [types.TYPE_CAFE] or [type.TYPE_BAR]
                # or [type.TYPE_CASINO])
                types =[types.TYPE_HOSPITAL])

        # If any attributions related
        # with search results print them
        if query_result.has_attributions: 
            print (query_result.html_attributions)

        
        # Iterate over the search results
        for place in query_result.places[:limit]:
            
            print(place)
            # place.get_details()
            print (place.name)

            print (place.name)
            place_geocoded = gmaps.reverse_geocode((place.geo_location['lat'], place.geo_location['lng']))
            if place_geocoded:
                result += f'Place Name : {place.name}\nAddress : {place_geocoded[0]["formatted_address"]}\nLatitude : {place.geo_location["lat"]}\nLongitude : {place.geo_location["lng"]}\n'
                print("Address",place_geocoded[0]["formatted_address"])
            else:
                result += f'Place Name : {place.name}\nAddress : Unavailable\nLatitude : {place.geo_location["lat"]}\nLongitude : {place.geo_location["lng"]}\n'
                print("Address Unvailable")
            print("Latitude", place.geo_location['lat'])
            print("Longitude", place.geo_location['lng'])
            print()

        return result

    else:
        result = "Incomplete Address! Please provide a Complete Address"
        print("Incomplete Address! Please provide a Complete Address")
        return result

    

In [59]:
result = get_nearest_hospital("24 south huntington ave boston",limit=5)

<Place name="Beth Israel Deaconess Medical Center", lat=42.33960629999999, lng=-71.10498199999999>
Beth Israel Deaconess Medical Center
Beth Israel Deaconess Medical Center
Address 330 Brookline Ave, Boston, MA 02215, USA
Latitude 42.33960629999999
Longitude -71.10498199999999

<Place name="Kerim Munir, MD, MPH, DSs", lat=42.3322788, lng=-71.11613729999999>
Kerim Munir, MD, MPH, DSs
Kerim Munir, MD, MPH, DSs
Address 2 Brookline Pl, Brookline, MA 02445, USA
Latitude 42.3322788
Longitude -71.11613729999999

<Place name="Jamaica Plain VA Medical Center", lat=42.32723349999999, lng=-71.11030359999999>
Jamaica Plain VA Medical Center
Jamaica Plain VA Medical Center
Address 150 S Huntington Ave, Boston, MA 02130, USA
Latitude 42.32723349999999
Longitude -71.11030359999999

<Place name="Mass General Brigham", lat=42.3473912, lng=-71.0820284>
Mass General Brigham
Mass General Brigham
Address 800 Boylston St Spc 101, Boston, MA 02199, USA
Latitude 42.3473912
Longitude -71.0820284

<Place name="